# Signal Events Information

Read the events file, which is generated using MadGraph-2.6.7 without any cuts.

In [1]:
import os
import random

import numpy as np
import ROOT as r
from matplotlib import pyplot as plt
r.gROOT.ProcessLine('.include /Users/isaac/Work/MG5_aMC_v2_6_7')
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/Delphes-3.4.2/external/ExRootAnalysis/ExRootTreeReader.h"')
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/ExRootAnalysis/ExRootClasses.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/ExRootAnalysis/ExRootLHEFReader.h"'
)
r.gSystem.Load(
    "/Users/isaac/Work/MG5_aMC_v2_6_7/ExRootAnalysis/libExRootAnalysis.so")

os.environ['TERM'] = 'linux'
random.seed(1)

Welcome to JupyROOT 6.22/00


## Add rootfiles

In [14]:
chain1 = r.TChain("LHEF")
chain1.Add("~/Work/MuonE/2to3sc/Events/nocut_0.001GeV/unweighted_events.root")
chain2 = r.TChain("LHEF")
chain2.Add("~/Work/MuonE/2to3sc/Events/nocut_0.005GeV/unweighted_events.root")
chain3 = r.TChain("LHEF")
chain3.Add("~/Work/MuonE/2to3sc/Events/nocut_0.01GeV/unweighted_events.root")
chain4 = r.TChain("LHEF")
chain4.Add("~/Work/MuonE/2to3sc/Events/nocut_0.05GeV/unweighted_events.root")
chain5 = r.TChain("LHEF")
chain5.Add("~/Work/MuonE/2to3sc/Events/nocut_0.1GeV/unweighted_events.root")
chainlist=[chain1, chain2, chain3, chain4, chain5]

## Basic settings
Here setup the luminosity, the cross section, and the elastic band.

In [15]:
x_section_list = [1.225e+07, 5.323e+06, 3.14e+06, 7.141e+05, 2.482e+05]
luminosity=1.5e+04
def T(theta_e):
    """T: defined as $E_4 - m_e$."""
    t = 2 * 0.000511 / (((150 + 0.000511)**2) /
                        ((np.cos(theta_e / 1000)**2) * (150**2 - 0.1057**2))
                        - 1)
    return t


def theta_mu_theory(theta_e):
    """The expression for the theta_mu vs theta_e relation in SM 2 to 2
    QED process"""
    theta = 1000 * np.arctan((np.sqrt(T(theta_e) * (2 * 0.000511 + T(theta_e)))
                              * np.sin(theta_e / 1000)) /
                             (np.sqrt(150**2 - 0.1057**2)
                              - np.sqrt(T(theta_e)
                                        * (2 * 0.000511 + T(theta_e)))
                              * np.cos(theta_e / 1000)))
    return theta

## Loop over everything
Loop over the files. For each file, we calculate no cuts, selection cuts, electron in ecal requirements and muon in ecal requirements. For the 40 targets at different positions, take average of them. 

In [31]:
for i in range(5):
    x_section = x_section_list[i]
    chain=chainlist[i]
    no_cuts = x_section * luminosity
    no_cuts = "{:.2e}".format(no_cuts)
    our_selection_number = 0
    our_and_electron_number = 0
    our_and_electron_and_muon_number = 0
    for event in chain:
        for particle in event.Particle:
            if(particle.Status == 1 and particle.PID == 13):
                muon_vector = r.TLorentzVector()  # muon 4-vector
                muon_vector.SetPtEtaPhiM(particle.PT, particle.Eta, particle.Phi, particle.M)
                theta_mu = muon_vector.Theta() * 1000 # theta_mu in mrad
                phi_mu = muon_vector.Phi() # phi_mu in rad
            elif(particle.Status == 1 and particle.PID == 11):
                electron_vector = r.TLorentzVector()
                electron_vector.SetPtEtaPhiM(particle.PT, particle.Eta, particle.Phi, particle.M)
                theta_e = electron_vector.Theta() * 1000 # theta_e in mrad
                phi_e = electron_vector.Phi()  # phi_e in rad
                E_e = particle.E
        if(E_e > 1.0 and theta_e <= 30.0 and theta_mu <= 5.0 and theta_mu > theta_mu_theory(theta_e) and abs(np.pi - abs(phi_e - phi_mu)) >= 3.5e-03):
            our_selection_number += 1
            for j in range(40):
                if(float(j)*theta_e/1000 <= 0.05):
                    our_and_electron_number += 1
                    if(float(j)*theta_mu/1000 <= 0.05):
                        our_and_electron_and_muon_number += 1

    our_selection = float(our_selection_number) * x_section * luminosity/10000
    our_selection = "{:2e}".format(our_selection)
    our_and_electron = float(our_and_electron_number) * x_section * luminosity/400000
    our_and_electron = "{:.2e}".format(our_and_electron)
    our_and_electron_and_muon = float(our_and_electron_and_muon_number) * x_section * luminosity/400000
    our_and_electron_and_muon = "{:.2e}".format(our_and_electron_and_muon)
    print('no cuts: ', no_cuts)
    print('our selection: ', our_selection)
    print('our selection + electron in ecal: ', our_and_electron)
    print('our selection + eletron and muon in ecal:', our_and_electron_and_muon)

('no cuts: ', '1.84e+11')
('our selection: ', '1.446112e+10')
('our selection + electron in ecal: ', '2.26e+09')
('our selection + eletron and muon in ecal:', '2.11e+09')
('no cuts: ', '7.98e+10')
('our selection: ', '7.641166e+09')
('our selection + electron in ecal: ', '1.12e+09')
('our selection + eletron and muon in ecal:', '1.06e+09')
('no cuts: ', '4.71e+10')
('our selection: ', '5.209260e+09')
('our selection + electron in ecal: ', '7.80e+08')
('our selection + eletron and muon in ecal:', '7.21e+08')
('no cuts: ', '1.07e+10')
('our selection: ', '1.782394e+09')
('our selection + electron in ecal: ', '2.86e+08')
('our selection + eletron and muon in ecal:', '2.66e+08')
('no cuts: ', '3.72e+09')
('our selection: ', '7.591197e+08')
('our selection + electron in ecal: ', '1.36e+08')
('our selection + eletron and muon in ecal:', '1.26e+08')
